In [63]:
from dotenv import load_dotenv
import base64

import os,sys

## openai libraries
from openai import OpenAI
import tiktoken

from pathlib import Path
import glob

import matplotlib.pyplot as plt
from PIL import Image
import cv2

import numpy as np

In [32]:
os.getcwd()

'/Users/ys/Work/01tutorials/openai'

In [34]:
# os.listdir("/Users/ys/Downloads")

In [110]:
load_dotenv(Path(os.getcwd()).parent / ".env",verbose=False)

True

In [98]:
download_folder = os.getenv("DOWNLOAD_FOLDER")

model = "gpt-4o-mini"

In [44]:
glob.glob(str(Path(download_folder) / "*.jpg"))

['/Users/ys/Downloads/25725218_m.jpg']

In [68]:
def encode_local_image_to_base64(image_path: str):
    with open(image_path,"rb") as f:
        image_base64 = base64.b64encode(f.read()).decode("utf-8")
    image_url = f"data:image/jpeg;base64,{image_base64}"
    return image_url

def imshow(img: np.array):
    f,ax = plt.subplots(figsize=(10,10))
    ax.imshow(img)
    plt.show()

def cv2_to_pillow(img: np.array):
    return Image.fromarray(img, mode='RGB')

## Prompt Engineering
* P48 ~ 

In [106]:
user_prompt = '''\
料理名:"""
{dish}
"""
'''

system_prompt = '''\
ユーザが入力した料理名のレシピを教えてください。
## 出力は以下のJSON形式にしてください。
```
{
 "材料" : ["材料1","材料2"],
 "手順" : ["手順1","手順2"],
}
```
'''

In [107]:
user_prompt

'料理名:"""\n{dish}\n"""\n'

In [108]:
def generate_recipe(dish: str) -> str:
    response = client.chat.completions.create(
        model = model,
        messages = [
            {
                "role" : "system",
                "content" : system_prompt
            },
            {
                "role" : "user",
                "content" : user_prompt.format(dish = dish)
            }
        ],
    )
    return response.choices[0].message.content

In [109]:
recipe = generate_recipe("カレー")
print(recipe)

{
 "材料" : ["米","カレールー","肉（鶏肉、豚肉、牛肉など）","玉ねぎ","にんじん","じゃがいも","水"],
 "手順" : ["米を洗って炊飯器で炊く。","鍋に油を熱し、切った玉ねぎを炒める。","肉を加えて炒め、色が変わったら切ったにんじんとじゃがいもを加える。","全体がよく混ざったら、水を加えて煮る。","煮立ったらアクを取り、カレールーを加えてさらに煮込む。","最後に味を整えて、ご飯の上にカレーをかけて盛り付ける。"]
}


In [74]:
local_image_path = '/Users/ys/Downloads/25725218_m.jpg'
local_image_path_down_sized = local_image_path.replace(".jpg","_downsized.jpg")

image_encode_url = encode_local_image_to_base64(local_image_path_down_sized)
print("base64の長さについて")
len(image_encode_url)

base64の長さについて


8711

In [75]:
img = np.array(Image.open(local_image_path))

print(img.shape)
img_resized = cv2.resize(img,(
    img.shape[1]//10,img.shape[0]//10
),interpolation=cv2.INTER_AREA)
print(img_resized.shape)

(1440, 1920, 3)
(144, 192, 3)


In [70]:
# imshow(img_resized)
# img_pillow = cv2_to_pillow(img_resized)
# img_pillow.save(local_image_path.replace(".jpg","_downsized.jpg"))

8711

In [16]:
text = "LLMを使ってクールなものを作るのは簡単だが、プロダクションで使えるものを作るのは非常に難しい。"

encoding = tiktoken.encoding_for_model("gpt-4o-mini")

In [17]:
tokens = encoding.encode(text)

In [20]:
# tokens

In [18]:
len(tokens)

37

In [22]:
client = OpenAI()

### 1. 画像の処理(ダウンサイズ済)

In [77]:
%%time

# response = client.chat.completions.create(
#     model = "gpt-4o-mini",
#     messages = [
#         {"role" : "system", "content":"You are a helpfull assistant."},
#         {"role" : "user", "content":"こんにちは、私はソリマチユーイチと言います！"},
#         {"role" : "assistant", "content":"こんにちは、ソリマチユーイチさん！今日は何を話しましょうか？"},
#         {"role" : "user", "content":"私の名前がわかりますか？"},
#     ],
# )

response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        # {
        #     "role" : "system", 
        #     "content":　"あなたは、画像を見て、何が書いてあるか説明する専門家です。"
        # },
        {
            "role" : "user",
            "content" : [
                { "type" : "text" , "text" : "画像を説明してください。" },
                { "type" : "image_url" , "image_url" : {"url":image_encode_url} },
            ]
        }
    ],
)

CPU times: user 11.9 ms, sys: 5.63 ms, total: 17.6 ms
Wall time: 3.12 s


In [84]:
response.choices[0].message.content

'画像には、ホッキョクグマが描かれています。ホッキョクグマは白い毛皮を持ち、通常は北極地域に生息しています。この画像では、ホッキョクグマが岩の上に立っている姿が見え、周囲には白い岩や水槽のようなものがあります。背景には自然の素材が使われているようです。'